In [1]:
import pandas as pd
import glob

/home/jaeyeonl/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid", palette="pastel")


In [3]:
aime24df = pd.read_parquet("hf://datasets/Maxwell-Jia/AIME_2024/aime_2024_problems.parquet")
aime24df = aime24df.rename(columns={'Problem': 'input', 'Answer': 'target'})
aime24df = aime24df[['input', 'target']]

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = aime24df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "total_tokens": df.token_usage.sum(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


aime_dfs = []
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minilow-aime24-msprt-1e61e6005094994-3/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-low")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minimedium-aime24-msprt-1e61e6005094994/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-medium")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-vote/*")).T.assign(method="Self-Cons@40", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-pvalue/*")).T.assign(method="p-value", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-bayesian/*")).T.assign(method="Ada-Cons", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="AIME24 / o3-mini-high")]
aime_dfs += [get_model_summary(get_model_problem_summary("../../aime24/o3minihigh-aime24-msprt-1e61e6005094994/*")).T.assign(method="mSPRT (Ours)", benchmark="AIME24 / o3-mini-high")]

pd.concat(aime_dfs)

,mean_trials,total_tokens,accuracy,method,benchmark
0,40.000000,1965052.0,0.700000,Self-Cons@40,AIME24 / o3-mini-low
0,16.400000,996432.0,0.700000,p-value,AIME24 / o3-mini-low
0,16.266667,1024628.0,0.700000,Ada-Cons,AIME24 / o3-mini-low
0,11.466667,708878.0,0.800000,SPRT (Ours),AIME24 / o3-mini-low
0,13.866667,839025.0,0.766667,mSPRT (Ours),AIME24 / o3-mini-low
0,40.000000,5990567.0,0.900000,Self-Cons@40,AIME24 / o3-mini-medium
0,11.200000,2384149.0,0.933333,p-value,AIME24 / o3-mini-medium
0,10.366667,2227098.0,0.900000,Ada-Cons,AIME24 / o3-mini-medium
0,4.733333,911649.0,0.900000,SPRT (Ours),AIME24 / o3-mini-medium
0,6.500000,1390765.0,0.900000,mSPRT (Ours),AIME24 / o3-mini-medium


In [4]:
import random
SEED = 42
random.seed(SEED)

df = pd.read_csv("../../data/gpqa_diamond.csv")

answer_columns = [
    "Correct Answer",
    "Incorrect Answer 1",
    "Incorrect Answer 2",
    "Incorrect Answer 3"
]

options_labels = ["A", "B", "C", "D"]

formatted_questions = []
correct_answers = []

for _, row in df.iterrows():
    shuffled_columns = random.sample(answer_columns, len(answer_columns))

    shuffled_options = {opt: row[col] for opt, col in zip(options_labels, shuffled_columns)}

    question_text = f"Question: {row['Pre-Revision Question']}\\n"
    choices_text = "\\n".join([f"{opt}. {shuffled_options[opt]}" for opt in options_labels])
    prompt_text = f"{question_text}{choices_text}"

    correct_option = [opt for opt, col in zip(options_labels, shuffled_columns) if col == "Correct Answer"][0]

    formatted_questions.append(prompt_text)
    correct_answers.append(correct_option)

gpqa_df = pd.DataFrame({
    "input": formatted_questions,
    "target": correct_answers
})
gpqa_df

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = gpqa_df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "total_tokens": df.token_usage.sum(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


gpqa_dfs = []
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_40_trail_multi_choice/*")).T.assign(method="Self-Cons@40", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_pvalue_V2/*")).T.assign(method="p-value", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_bayesian_posterior_multi_choice/*")).T.assign(method="Ada-Cons", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_sprt_V2/*")).T.assign(method="SPRT (Ours)", benchmark="GPQA Diamond / o3-mini-low")]
gpqa_dfs += [get_model_summary(get_model_problem_summary("../../gpqa_diamond/gpqa_msprt_V2/*")).T.assign(method="mSPRT (Ours)", benchmark="GPQA Diamond / o3-mini-low")]
pd.concat(gpqa_dfs)


,mean_trials,total_tokens,accuracy,method,benchmark
0,40.000000,3919554.0,0.681818,Self-Cons@40,GPQA Diamond / o3-mini-low
0,11.378788,1443465.0,0.691919,p-value,GPQA Diamond / o3-mini-low
0,8.878788,1173271.0,0.671717,Ada-Cons,GPQA Diamond / o3-mini-low
0,4.707071,540879.0,0.666667,SPRT (Ours),GPQA Diamond / o3-mini-low
0,5.121212,641129.0,0.702020,mSPRT (Ours),GPQA Diamond / o3-mini-low


In [5]:
gsm_df = pd.read_json('../../data/gsm.jsonl', lines=True)

def get_model_problem_summary(globpath):
    df = pd.concat([pd.read_csv(filepath, index_col=0).assign(id=int(filepath.split("/")[-1].split('.')[0])) for filepath in sorted(glob.glob(globpath))]).reset_index(drop=True)
    df2 = gsm_df.target.to_frame()
    df2['answer'] = df.groupby('id').answer.agg(lambda x: x.value_counts().index[0])
    df2['trials'] = df.groupby('id').token_usage.count()
    df2['token_usage'] = df.groupby('id').token_usage.sum()
    # print(df2)
    return df2

def get_model_summary(df):
    return pd.Series({
        "mean_trials": df.trials.mean(),
        "total_tokens": df.token_usage.sum(),
        "accuracy": (df.target == df.answer).mean(),
    }).to_frame()


gsm_dfs = []
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-vote/*")).T.assign(method="Self-Cons@40", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-pvalue/*")).T.assign(method="p-value", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-bayesian/*")).T.assign(method="Ada-Cons", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-sprt-050010050949976/*")).T.assign(method="SPRT (Ours)", benchmark="GSM / gpt-4o-mini")]
gsm_dfs += [get_model_summary(get_model_problem_summary("../../gsm/gpt4omini-gsm-msprt-1e61e6005094994/*")).T.assign(method="mSPRT (Ours)", benchmark="GSM / gpt-4o-mini")]
pd.concat(gsm_dfs)


,mean_trials,total_tokens,accuracy,method,benchmark
0,40.000000,7004327.0,0.941622,Self-Cons@40,GSM / gpt-4o-mini
0,7.872631,1453838.0,0.941622,p-value,GSM / gpt-4o-mini
0,6.399545,1201976.0,0.941622,Ada-Cons,GSM / gpt-4o-mini
0,4.352540,801760.0,0.934799,SPRT (Ours),GSM / gpt-4o-mini
0,4.288855,791509.0,0.938590,mSPRT (Ours),GSM / gpt-4o-mini


In [6]:
df2 = pd.concat(aime_dfs + gpqa_dfs + gsm_dfs).pivot_table(values=["accuracy", "mean_trials", "total_tokens"], columns="method", index="benchmark", sort=False)
df2 = df2.rename(columns={"accuracy": "Accuracy", "mean_trials": "Num. Trials", "total_tokens": "Token Usage"})
df2.index = df2.index.set_names("Benchmark / LLM")
df2.columns = df2.columns.set_names([None, "Method"])
df2

Num. Trials                                    \
Method                     Self-Cons@40    p-value   Ada-Cons SPRT (Ours)   
Benchmark / LLM                                                             
AIME24 / o3-mini-low               40.0  16.400000  16.266667   11.466667   
AIME24 / o3-mini-medium            40.0  11.200000  10.366667    4.733333   
AIME24 / o3-mini-high              40.0   9.733333   7.400000    4.133333   
GPQA Diamond / o3-mini-low         40.0  11.378788   8.878788    4.707071   
GSM / gpt-4o-mini                  40.0   7.872631   6.399545    4.352540   

                                         Token Usage                        \
Method                     mSPRT (Ours) Self-Cons@40    p-value   Ada-Cons   
Benchmark / LLM                                                              
AIME24 / o3-mini-low          13.866667    1965052.0   996432.0  1024628.0   
AIME24 / o3-mini-medium        6.500000    5990567.0  2384149.0  2227098.0   
AIME24 / o3-mini-high          4.366667   12898010.0  5932607.0  4566309.0   
GPQA Diamond / o3-mini-low     5.121212    3919554.0  1443465.0  1173271.0   
GSM / gpt-4o-mini              4.288855    7004327.0  1453838.0  1201976.0   

                                                        Accuracy            \
Method                     SPRT (Ours) mSPRT (Ours) Self-Cons@40   p-value   
Benchmark / LLM                                                              
AIME24 / o3-mini-low          708878.0     839025.0     0.700000  0.700000   
AIME24 / o3-mini-medium       911649.0    1390765.0     0.900000  0.933333   
AIME24 / o3-mini-high        1999343.0    2298738.0     0.966667  0.966667   
GPQA Diamond / o3-mini-low    540879.0     641129.0     0.681818  0.691919   
GSM / gpt-4o-mini             801760.0     791509.0     0.941622  0.941622   

                                                               
Method                      Ada-Cons SPRT (Ours) mSPRT (Ours)  
Benchmark / LLM                                                
AIME24 / o3-mini-low        0.700000    0.800000     0.766667  
AIME24 / o3-mini-medium     0.900000    0.900000     0.900000  
AIME24 / o3-mini-high       0.966667    0.966667     0.966667  
GPQA Diamond / o3-mini-low  0.671717    0.666667     0.702020  
GSM / gpt-4o-mini           0.941622    0.934799     0.938590

In [7]:
print((df2["Accuracy"]*100).to_latex(float_format="%.1f\\%%"))
print(df2["Num. Trials"].to_latex(float_format="%.2f"))
print((100 - df2["Token Usage"].T / df2["Token Usage"]["Self-Cons@40"] * 100).T.to_latex(float_format="%.1f\\%%"))

\begin{tabular}{lrrrrr}
\toprule
Method & Self-Cons@40 & p-value & Ada-Cons & SPRT (Ours) & mSPRT (Ours) \\
Benchmark / LLM &  &  &  &  &  \\
\midrule
AIME24 / o3-mini-low & 70.0\% & 70.0\% & 70.0\% & 80.0\% & 76.7\% \\
AIME24 / o3-mini-medium & 90.0\% & 93.3\% & 90.0\% & 90.0\% & 90.0\% \\
AIME24 / o3-mini-high & 96.7\% & 96.7\% & 96.7\% & 96.7\% & 96.7\% \\
GPQA Diamond / o3-mini-low & 68.2\% & 69.2\% & 67.2\% & 66.7\% & 70.2\% \\
GSM / gpt-4o-mini & 94.2\% & 94.2\% & 94.2\% & 93.5\% & 93.9\% \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
Method & Self-Cons@40 & p-value & Ada-Cons & SPRT (Ours) & mSPRT (Ours) \\
Benchmark / LLM &  &  &  &  &  \\
\midrule
AIME24 / o3-mini-low & 40.00 & 16.40 & 16.27 & 11.47 & 13.87 \\
AIME24 / o3-mini-medium & 40.00 & 11.20 & 10.37 & 4.73 & 6.50 \\
AIME24 / o3-mini-high & 40.00 & 9.73 & 7.40 & 4.13 & 4.37 \\
GPQA Diamond / o3-mini-low & 40.00 & 11.38 & 8.88 & 4.71 & 5.12 \\
GSM / gpt-4o-mini & 40.00 & 7.87 & 6.40 & 4.35 & 4.29 \\
\bott

> **Note**: Copy the above result and put \bf to the remark our result.